In [2]:
import pandas as pd

In [4]:
df = pd.read_excel('newest-1.xlsx')

In [5]:
df.head()

,Marine Domain,Village Or Water Station,Date,latitude,longitude
0,"چاریکار""غوربند""",چاریکار,2022-05-03,35.01361,69.17139
1,دریایی کابل( وسطی),دريای پغمان,2022-05-03,34.50000,69.15
2,دریایی کابل( وسطی),ده سبز,2022-05-03,34.66555,69.308
3,دریایی کابل( وسطی),گل غوندی,2022-05-04,35.01914,69.15114
4,دریایی کابل( وسطی),دريای پغمان,2022-05-04,34.50000,69.15


In [32]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Function to convert the date into the API format
def convert_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d")

# Function to fetch precipitation data from Open-Meteo API for a given latitude, longitude, and date
def fetch_precipitation_data(lat, lon, start_date, end_date):
    try:
        # Convert latitude and longitude to float for validation
        lat = float(lat)
        lon = float(lon)
    except ValueError:
        print(f"Invalid coordinates: latitude={lat}, longitude={lon}. Skipping this row.")
        return None
    
    # Validate latitude and longitude
    if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
        print(f"Invalid coordinates: latitude={lat}, longitude={lon}. Skipping this row.")
        return None

    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "daily": "precipitation_sum",
        "timezone": "auto"
    }
    
    try:
        # Send the request to the Open-Meteo API
        responses = openmeteo.weather_api(url, params=params)
        return responses
    except Exception as e:
        print(f"Error fetching data for {lat}, {lon}, {start_date}: {e}")
        return None

# Load your dataset (adjust with your actual Excel file path)
input_excel_file = "newest-1.xlsx"  # Replace with your dataset file path
df = pd.read_excel(input_excel_file)

# Add a new column for precipitation
df['precipitation'] = None

# Iterate over each row in your dataset
for index, row in df.iterrows():
    date = row['Date']  # Assuming the column name is 'Date'
    lat = row['latitude']  # Assuming the column name is 'latitude'
    lon = row['longitude']  # Assuming the column name is 'longitude'
    
    # Convert the date to the appropriate format for the API
    start_date = convert_date(str(date))  # Convert to YYYY-MM-DD
    end_date = start_date  # For single day data, end_date is the same as start_date
    
    # Fetch the precipitation data for this date and location
    responses = fetch_precipitation_data(lat, lon, start_date, end_date)
    
    if responses:
        # Assuming that the response contains daily precipitation data
        response = responses[0]
        daily = response.Daily()
        daily_precipitation_sum = daily.Variables(0).ValuesAsNumpy()

        # Extract the daily precipitation sum for the day
        daily_precipitation_value = daily_precipitation_sum[0] if len(daily_precipitation_sum) > 0 else None
        df.at[index, 'precipitation'] = daily_precipitation_value
        print(f"Precipitation data saved for {lat}, {lon}, {start_date}: {daily_precipitation_value}")
    else:
        print(f"Skipping row {index}: No precipitation data found for {lat}, {lon}, {start_date}")

# Save the updated dataset to a new Excel file
output_excel_file = "output_dataset_with_precipitations.xlsx"  # Output file name
df.to_excel(output_excel_file, index=False)

print(f"Updated dataset saved to {output_excel_file}")


Precipitation data saved for 35.01361, 69.17139, 2022-05-03: 15.100000381469727
Precipitation data saved for 34.5, 69.15, 2022-05-03: 4.099999904632568
Precipitation data saved for 34.66555, 69.308, 2022-05-03: 10.399999618530273
Precipitation data saved for 35.01914, 69.15114, 2022-05-04: 14.600001335144043
Precipitation data saved for 34.5, 69.15, 2022-05-04: 6.899999618530273
Precipitation data saved for 32.41667, 68.75, 2022-06-07: 0.0
Precipitation data saved for 34.77472, 111.18139, 2022-07-04: 0.0
Precipitation data saved for 32.89864, 69.27371, 2022-07-04: 2.1000001430511475
Precipitation data saved for 33.10647, 69.05464, 2022-07-04: 0.6000000238418579
Precipitation data saved for 32.99863, 68.77225, 2022-07-04: 0.0
Precipitation data saved for 33.42906, 68.30439, 2022-07-04: 0.0
Precipitation data saved for 33.75764, 68.91321, 2022-07-05: 12.0
Precipitation data saved for 32.89864, 69.27371, 2022-07-05: 2.799999952316284
Precipitation data saved for 33.10647, 69.05464, 2022-0

In [33]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Function to convert the date into the API format
def convert_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d")

# Function to fetch soil moisture data from Open-Meteo API for a given latitude, longitude, and date
def fetch_soil_moisture_data(lat, lon, start_date, end_date):
    try:
        # Convert latitude and longitude to float for validation
        lat = float(lat)
        lon = float(lon)
    except ValueError:
        print(f"Invalid coordinates: latitude={lat}, longitude={lon}. Skipping this row.")
        return None
    
    # Validate latitude and longitude
    if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
        print(f"Invalid coordinates: latitude={lat}, longitude={lon}. Skipping this row.")
        return None

    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": "soil_moisture_0_to_7cm",  # Soil moisture data
        "timezone": "auto"
    }
    
    try:
        # Send the request to the Open-Meteo API
        responses = openmeteo.weather_api(url, params=params)
        return responses
    except Exception as e:
        print(f"Error fetching data for {lat}, {lon}, {start_date}: {e}")
        return None

# Load your dataset (adjust with your actual Excel file path)
input_excel_file = "newest-1.xlsx"  # Replace with your dataset file path
df = pd.read_excel(input_excel_file)

# Add a new column for soil moisture
df['soil_moisture_0_to_7cm'] = None

# Iterate over each row in your dataset
for index, row in df.iterrows():
    date = row['Date']  # Assuming the column name is 'Date'
    lat = row['latitude']  # Assuming the column name is 'latitude'
    lon = row['longitude']  # Assuming the column name is 'longitude'
    
    # Convert the date to the appropriate format for the API
    start_date = convert_date(str(date))  # Convert to YYYY-MM-DD
    end_date = start_date  # For single day data, end_date is the same as start_date
    
    # Fetch the soil moisture data for this date and location
    responses = fetch_soil_moisture_data(lat, lon, start_date, end_date)
    
    if responses:
        # Assuming that the response contains hourly soil moisture data
        response = responses[0]
        hourly = response.Hourly()
        hourly_soil_moisture_0_to_7cm = hourly.Variables(0).ValuesAsNumpy()

        # Extract the hourly soil moisture value for the day
        hourly_soil_moisture_value = hourly_soil_moisture_0_to_7cm[0] if len(hourly_soil_moisture_0_to_7cm) > 0 else None
        df.at[index, 'soil_moisture_0_to_7cm'] = hourly_soil_moisture_value
        print(f"Soil moisture data saved for {lat}, {lon}, {start_date}: {hourly_soil_moisture_value}")
    else:
        print(f"Skipping row {index}: No soil moisture data found for {lat}, {lon}, {start_date}")

# Save the updated dataset to a new Excel file
output_excel_file = "output_dataset_with_soil_moisture.xlsx"  # Output file name
df.to_excel(output_excel_file, index=False)

print(f"Updated dataset saved to {output_excel_file}")


Soil moisture data saved for 35.01361, 69.17139, 2022-05-03: 0.20900000631809235
Soil moisture data saved for 34.5, 69.15, 2022-05-03: 0.18299999833106995
Soil moisture data saved for 34.66555, 69.308, 2022-05-03: 0.16200000047683716
Soil moisture data saved for 35.01914, 69.15114, 2022-05-04: 0.3880000114440918
Soil moisture data saved for 34.5, 69.15, 2022-05-04: 0.27300000190734863
Soil moisture data saved for 32.41667, 68.75, 2022-06-07: 0.017000000923871994
Soil moisture data saved for 34.77472, 111.18139, 2022-07-04: 0.16699999570846558
Soil moisture data saved for 32.89864, 69.27371, 2022-07-04: 0.014999999664723873
Soil moisture data saved for 33.10647, 69.05464, 2022-07-04: 0.023000000044703484
Soil moisture data saved for 32.99863, 68.77225, 2022-07-04: 0.023000000044703484
Soil moisture data saved for 33.42906, 68.30439, 2022-07-04: 0.023000000044703484
Soil moisture data saved for 33.75764, 68.91321, 2022-07-05: 0.11599999666213989
Soil moisture data saved for 32.89864, 69.

In [36]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Function to convert the date into the API format
def convert_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d")

# Function to fetch snow depth and snowfall data from Open-Meteo API for a given latitude, longitude, and date
def fetch_weather_data(lat, lon, start_date, end_date):
    try:
        # Convert latitude and longitude to float for validation
        lat = float(lat)
        lon = float(lon)
    except ValueError:
        print(f"Invalid coordinates: latitude={lat}, longitude={lon}. Skipping this row.")
        return None

    # Validate latitude and longitude
    if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
        print(f"Invalid coordinates: latitude={lat}, longitude={lon}. Skipping this row.")
        return None

    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": "snow_depth",
        "daily": "snowfall_sum",
        "timezone": "auto"
    }

    try:
        # Send the request to the Open-Meteo API
        responses = openmeteo.weather_api(url, params=params)
        return responses
    except Exception as e:
        print(f"Error fetching data for {lat}, {lon}, {start_date}: {e}")
        return None

# Load your dataset (adjust with your actual Excel file path)
input_excel_file = "newest-1.xlsx"  # Replace with your dataset file path
df = pd.read_excel(input_excel_file)

# Add new columns for hourly snow depth and daily snowfall sum
df['hourly_snow_depth'] = None
df['daily_snowfall_sum'] = None

# Iterate over each row in your dataset
for index, row in df.iterrows():
    date = row['Date']  # Assuming the column name is 'Date'
    lat = row['latitude']  # Assuming the column name is 'latitude'
    lon = row['longitude']  # Assuming the column name is 'longitude'

    # Convert the date to the appropriate format for the API
    start_date = convert_date(str(date))  # Convert to YYYY-MM-DD
    end_date = start_date  # For single-day data, end_date is the same as start_date

    # Fetch the weather data for this date and location
    responses = fetch_weather_data(lat, lon, start_date, end_date)

    if responses:
        # Process the first response for hourly and daily data
        response = responses[0]

        # Extract hourly snow depth
        hourly = response.Hourly()
        hourly_snow_depth = hourly.Variables(0).ValuesAsNumpy()
        hourly_snow_depth_value = hourly_snow_depth[0] if len(hourly_snow_depth) > 0 else None

        # Extract daily snowfall sum
        daily = response.Daily()
        daily_snowfall_sum = daily.Variables(0).ValuesAsNumpy()
        daily_snowfall_value = daily_snowfall_sum[0] if len(daily_snowfall_sum) > 0 else None

        # Save data to the DataFrame
        df.at[index, 'hourly_snow_depth'] = hourly_snow_depth_value
        df.at[index, 'daily_snowfall_sum'] = daily_snowfall_value
        print(f"Data saved for {lat}, {lon}, {start_date}: Snow Depth = {hourly_snow_depth_value}, Snowfall = {daily_snowfall_value}")
    else:
        print(f"Skipping row {index}: No data found for {lat}, {lon}, {start_date}")

# Save the updated dataset to a new Excel file
output_excel_file = "output_weather_data.xlsx"  # Output file name
df.to_excel(output_excel_file, index=False)

print(f"Updated dataset saved to {output_excel_file}")


Data saved for 35.01361, 69.17139, 2022-05-03: Snow Depth = 0.0, Snowfall = 0.0
Data saved for 34.5, 69.15, 2022-05-03: Snow Depth = 0.0, Snowfall = 0.0
Data saved for 34.66555, 69.308, 2022-05-03: Snow Depth = 0.0, Snowfall = 0.0
Data saved for 35.01914, 69.15114, 2022-05-04: Snow Depth = 0.0, Snowfall = 0.0
Data saved for 34.5, 69.15, 2022-05-04: Snow Depth = 0.0, Snowfall = 0.0
Data saved for 32.41667, 68.75, 2022-06-07: Snow Depth = 0.0, Snowfall = 0.0
Data saved for 34.77472, 111.18139, 2022-07-04: Snow Depth = 0.0, Snowfall = 0.0
Data saved for 32.89864, 69.27371, 2022-07-04: Snow Depth = 0.0, Snowfall = 0.0
Data saved for 33.10647, 69.05464, 2022-07-04: Snow Depth = 0.0, Snowfall = 0.0
Data saved for 32.99863, 68.77225, 2022-07-04: Snow Depth = 0.0, Snowfall = 0.0
Data saved for 33.42906, 68.30439, 2022-07-04: Snow Depth = 0.0, Snowfall = 0.0
Data saved for 33.75764, 68.91321, 2022-07-05: Snow Depth = 0.0, Snowfall = 0.0
Data saved for 32.89864, 69.27371, 2022-07-05: Snow Depth 

KeyboardInterrupt: 

In [1]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Function to convert the date into the API format
def convert_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d")

# Function to fetch river discharge data from Open-Meteo API for a given latitude, longitude, and date
def fetch_weather_data(lat, lon, start_date, end_date):
    try:
        # Convert latitude and longitude to float for validation
        lat = float(lat)
        lon = float(lon)
    except ValueError:
        print(f"Invalid coordinates: latitude={lat}, longitude={lon}. Skipping this row.")
        return None

    # Validate latitude and longitude
    if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
        print(f"Invalid coordinates: latitude={lat}, longitude={lon}. Skipping this row.")
        return None

    url = "https://flood-api.open-meteo.com/v1/flood"
    params = {
        "latitude": lat,
        "longitude": lon,
        "daily": "river_discharge",
        "timezone": "auto"
    }

    try:
        # Send the request to the Open-Meteo API
        responses = openmeteo.weather_api(url, params=params)
        return responses
    except Exception as e:
        print(f"Error fetching data for {lat}, {lon}, {start_date}: {e}")
        return None

# Load your dataset (adjust with your actual Excel file path)
input_excel_file = "newest-1.xlsx"  # Replace with your dataset file path
df = pd.read_excel(input_excel_file)

# Add new columns for daily river discharge
df['daily_river_discharge'] = None

# Iterate over each row in your dataset
for index, row in df.iterrows():
    date = row['Date']  # Assuming the column name is 'Date'
    lat = row['latitude']  # Assuming the column name is 'latitude'
    lon = row['longitude']  # Assuming the column name is 'longitude'

    # Convert the date to the appropriate format for the API
    start_date = convert_date(str(date))  # Convert to YYYY-MM-DD
    end_date = start_date  # For single-day data, end_date is the same as start_date

    # Fetch the weather data for this date and location
    responses = fetch_weather_data(lat, lon, start_date, end_date)

    if responses:
        # Process the first response for daily river discharge data
        response = responses[0]

        # Extract daily river discharge
        daily = response.Daily()
        daily_river_discharge = daily.Variables(0).ValuesAsNumpy()
        daily_river_discharge_value = daily_river_discharge[0] if len(daily_river_discharge) > 0 else None

        # Save data to the DataFrame
        df.at[index, 'daily_river_discharge'] = daily_river_discharge_value
        print(f"Data saved for {lat}, {lon}, {start_date}: River Discharge = {daily_river_discharge_value}")
    else:
        print(f"Skipping row {index}: No data found for {lat}, {lon}, {start_date}")

# Save the updated dataset to a new Excel file
output_excel_file = "output_weather_data.xlsx"  # Output file name
df.to_excel(output_excel_file, index=False)

print(f"Updated dataset saved to {output_excel_file}")


Data saved for 35.01361, 69.17139, 2022-05-03: River Discharge = 0.04712855815887451
Data saved for 34.5, 69.15, 2022-05-03: River Discharge = 0.3899526596069336
Data saved for 34.66555, 69.308, 2022-05-03: River Discharge = 0.0
Data saved for 35.01914, 69.15114, 2022-05-04: River Discharge = 0.04712855815887451
Data saved for 34.5, 69.15, 2022-05-04: River Discharge = 0.3899526596069336
Data saved for 32.41667, 68.75, 2022-06-07: River Discharge = 0.0
Data saved for 34.77472, 111.18139, 2022-07-04: River Discharge = 834.6029052734375
Data saved for 32.89864, 69.27371, 2022-07-04: River Discharge = 0.0
Data saved for 33.10647, 69.05464, 2022-07-04: River Discharge = 0.0
Data saved for 32.99863, 68.77225, 2022-07-04: River Discharge = 0.0
Data saved for 33.42906, 68.30439, 2022-07-04: River Discharge = 0.0
Data saved for 33.75764, 68.91321, 2022-07-05: River Discharge = 0.0
Data saved for 32.89864, 69.27371, 2022-07-05: River Discharge = 0.0
Data saved for 33.10647, 69.05464, 2022-07-05